In [1]:
import pandas as pd
import os

np = pd.np

data_dir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games/Data/4_rcn')

trained_files = [f for f in os.listdir(os.path.join(data_dir, 'Trained_Game')) if f[-3:]=='csv']
untrained_files = [f for f in os.listdir(os.path.join(data_dir, 'Untrained_Game')) if f[-3:]=='csv']

% matplotlib inline

In [2]:
def reconstruct_boards(grouped):
    """Reconstructs a trial using the move/color records"""
    bp, wp = ('0'*36, ) * 2
    
    new_pos = []
    for i in grouped.index.values:
        row = grouped.loc[i]
        color, zet = row[['color', 'zet']]
        
        if color == 0:
            p = bp
            op = wp
        else:
            p = wp
            op = bp
            
        newp = list(p)
        newop = list(op)
        
        if newp[zet] == '1':
            newp[zet] = '0'
        elif newop[zet] == '1':
            newop[zet] = '0'
            newp[zet] = '1'
        else:
            newp[zet] = '1'
            
        if color == 0:
            bp = ''.join(newp)
            wp = ''.join(newop)
        else:
            wp = ''.join(newp)
            bp = ''.join(newop)
            
        new_pos.append((bp, wp))
        
    return pd.DataFrame(new_pos, columns=['bp', 'wp'])

def clean_reconstruction_data(filename, trained=True):
    
    # 0: Load data, correct game index field
    names = ['idx', 'subid', 'color', 'gi', 'mi', 'status', 'bp', 'wp', 'zet', 'rt', 'ts', 'mt', 'mx']
    directory = 'Trained_Game' if trained else 'Untrained_Game'
    df = pd.read_csv(os.path.join(data_dir, directory, filename), names=names)
    df['subject'] = filename.split('_')[1][:-4]
    df['trained'] = trained
    reconi = df['status']=='reconi'
    df['gi'] = np.nan
    df.loc[reconi, 'gi'] = np.arange(len(df.loc[reconi]))
    df['gi'] = df['gi'].fillna(method='ffill', downcast='infer')
    
    
    # 1: Boards recorded "0" inputs wrong; reconstruct them from moves
    reconm = df['status'] == 'recon'
    reconf = df['status'] == 'reconf'
    recond = df.loc[reconm].groupby('gi').apply(reconstruct_boards)
    assembled = recond.reset_index()

    df['bp correct'] = df['bp']
    df['wp correct'] = df['wp']
    df.loc[reconm, 'bp correct'] = assembled['bp'].values
    df.loc[reconm, 'wp correct'] = assembled['wp'].values
    df.loc[reconf, 'bp correct'] = np.nan
    df.loc[reconf, 'wp correct'] = np.nan
    df['bp correct'] = df['bp correct'].fillna(method='ffill')
    df['wp correct'] = df['wp correct'].fillna(method='ffill')

    # 2: Response times are goofed; reconstruct from timestamps
    df['rt correct'] = df['ts'] - df['ts'].shift(1)
    df.loc[df['status'] == 'eyecal', 'rt correct'] = df.loc[df['status'] == 'eyecal', 'rt']
    
    # 3: Rename and drop columns (this can be combined above when done debugging)
    df['rt'] = df['rt correct']
    board_filter = df['status'].isin(['recon', 'reconf'])
    df.loc[board_filter, 'bp'] = df.loc[board_filter, 'bp correct']
    df.loc[board_filter, 'wp'] = df.loc[board_filter, 'wp correct']
    df = df[['subject', 'trained', 'color', 'gi', 'mi', 'status', 'bp', 'wp', 'zet', 'rt', 'ts']]

    return df

for trained in trained_files:
    df = clean_reconstruction_data(trained, trained=True)
    filename = 'trained {subject}.csv'.format(subject=trained.split('_')[1][:-4])
    df.to_csv(os.path.join(data_dir, 'Clean/Game', filename), index=False)

for untrained in untrained_files:
    df = clean_reconstruction_data(untrained, trained=False)
    filename = 'untrained {subject}.csv'.format(subject=untrained.split('_')[1][:-4])
    df.to_csv(os.path.join(data_dir, 'Clean/Game', filename), index=False)

In [3]:
df = clean_reconstruction_data(untrained_files[-3], trained=False)

idxs = df.loc[df['rt'] < 0].index.values
idxs = pd.np.concatenate([idxs, idxs - 1])
df.loc[idxs].sort_index()

,subject,trained,color,gi,mi,status,bp,wp,zet,rt,ts
145,SR,False,0,5.0,11,recon,100000000000000000010000000000000000,000100000001000000000110000000110000,0,807.0,1469632027556
146,SR,False,0,5.0,10,recon,100000000001000000010000000000000000,000100000000000000000110000000110000,11,-2.0,1469632027554
236,SR,False,1,11.0,3,recon,000010000000000000000000000000000000,100100000000000000000000000000000000,0,932.0,1469632195102
237,SR,False,1,11.0,2,recon,000010000000000000000000000000000000,100000000000000000000000000000000000,3,-2.0,1469632195100
574,SR,False,1,28.0,6,recon,000000000000001000000001000011001000,100000000000000000000000000000000000,0,622.0,1469632731337
575,SR,False,1,28.0,5,recon,000000000000000000000001000011001000,100000000000001000000000000000000000,14,-2.0,1469632731335
771,SR,False,0,37.0,13,recon,100001000000000100000001000000000000,000010000000100000001110000000010000,0,832.0,1469633076391
772,SR,False,0,37.0,12,recon,100001000000000100001001000000000000,000010000000100000000110000000010000,20,-2.0,1469633076389
996,SR,False,1,48.0,9,recon,000000000000000000000100000010100000,110000000010000000000000000000000000,0,2348.0,1469633423345
997,SR,False,1,48.0,8,recon,000000000000000000000000000010100000,110000000010000000000100000000000000,21,-2.0,1469633423343
